In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"#d_model必须是num_heads的整数倍，如果不是就会报错
        
        self.d_model = d_model#模型的维度
        self.num_heads = num_heads#头的数量
        self.d_k = d_model // num_heads#每个头的维度
        
        self.W_q = nn.Linear(d_model, d_model)#定义Q矩阵
        self.W_k = nn.Linear(d_model, d_model)#定义K矩阵
        self.W_v = nn.Linear(d_model, d_model)#定义V矩阵
        self.W_o = nn.Linear(d_model, d_model)#定义输出矩阵
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [3]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [4]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [5]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [6]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [7]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [8]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [13]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(20):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")



Epoch: 1, Loss: 4.904193878173828
Epoch: 2, Loss: 6.176750183105469
Epoch: 3, Loss: 5.102799892425537
Epoch: 4, Loss: 5.019215106964111
Epoch: 5, Loss: 4.968686580657959
Epoch: 6, Loss: 4.8707661628723145
Epoch: 7, Loss: 4.801056385040283
Epoch: 8, Loss: 4.726855754852295
Epoch: 9, Loss: 4.667356967926025
Epoch: 10, Loss: 4.618838310241699
Epoch: 11, Loss: 4.571394920349121
Epoch: 12, Loss: 4.534541130065918
Epoch: 13, Loss: 4.480047225952148
Epoch: 14, Loss: 4.442187786102295
Epoch: 15, Loss: 4.3917412757873535
Epoch: 16, Loss: 4.347477436065674
Epoch: 17, Loss: 4.297293186187744
Epoch: 18, Loss: 4.2492899894714355
Epoch: 19, Loss: 4.201030254364014
Epoch: 20, Loss: 4.156561851501465


In [15]:
transformer.eval()  # 设置为评估模式

# 取一个源序列样本进行推理
src_sample = src_data[1].unsqueeze(0)  # 形状: (1, max_seq_length)

# 初始化目标序列，从 <bos>（开始标记）开始
bos_token = 1  # 假设 1 是开始标记 <bos>
eos_token = 2  # 假设 2 是结束标记 <eos>
tgt_sample = torch.LongTensor([[bos_token]])  # 形状: (1, 1)

# 逐词生成目标序列
max_gen_length = 50  # 设置最大生成长度
with torch.no_grad():  # 禁用梯度计算
    for _ in range(max_gen_length):
        # 前向传播，生成下一个词的概率
        output = transformer(src_sample, tgt_sample)
        # 取最后一个位置的输出，形状: (1, vocab_size)
        next_word_logits = output[:, -1, :]
        # 选择概率最高的词
        next_word = torch.argmax(next_word_logits, dim=-1)  # 形状: (1,)
        
        # 将预测的词添加到目标序列
        tgt_sample = torch.cat([tgt_sample, next_word.unsqueeze(1)], dim=1)  # 拼接
        
        # 如果生成结束标记 <eos>，停止
        if next_word.item() == eos_token:
            break

# 打印生成的序列
print("Source sequence:", src_sample)
print("Generated target sequence:", tgt_sample)

Source sequence: tensor([[1306, 4716, 1232, 1979,    6, 4749, 1985, 2218,   12,   13, 2541, 2048,
          904, 4593,  899, 2060, 2767, 1569, 4126, 1376, 1055, 2424,  125,  128,
         1952, 2177, 3319, 4779, 2775,  918, 1457, 4355,  101, 1269, 3015,  874,
         4748, 1442,  193, 4657, 3686, 1813, 1277,  405, 1922, 1684,  910, 3779,
         3135,  975, 1763, 3523,  537, 1848, 1212,  920, 3280, 2097,  338, 4059,
          396,  471, 3890, 1998,   74,  825, 2872,  720,  989,  997, 3444, 1395,
         3915, 2753,  516,  609, 3275, 3080, 2644, 2164, 2588, 3566, 1129,  453,
         1353, 4140,  648,  336, 4588, 1655, 4996, 4536, 4513, 1996,  715, 3710,
         2624, 2486, 2481, 2047]])
Generated target sequence: tensor([[   1, 3775, 3831,  651, 1394, 1590, 1961,  238, 1541, 2283, 1076, 2650,
         1268, 3987,  317, 4856,  545, 4936, 4684, 1619, 2240, 3757, 1429, 4238,
         2653, 2553, 3134, 4355, 4955, 3336,  672, 1738, 2027,  565, 2791,   77,
         1927, 2851,  299, 397